In [3]:
import EcoFOCIpy.plots.sbe_ctd_plots as sbe_ctd_plots
import os
import xarray as xa
import seaborn as sns
import numpy as np

In [13]:
sample_data_dir = '/Users/bell/ecoraid/2010/CTDcasts/we1008/erddap/final_data/ctd/' #root path to cruise directory working repoc

### QC of data (plot parameters with other instruments)
- be sure to updated the qc_status for the file and the history for future steps
- from NRT csv files... make edits then use that to modify netcdf?
- **Better Yet** export netcdf files with variable flags to csv and edit then reingest and modify
    - modify netcdf in place and provide a qcflag?


### Likely first edits
- research issues noted in cruise logs
- remove failed instrument (clogs, or other challenges that arent correctable)
- extrapolate to sfc (not par though)
- despike depths (interpolate between depths)
- check for common issues
    - lags in salinity when going through interface

In [14]:
#test for salinity bottle run needs
for cast in sorted(os.listdir(sample_data_dir)):
    if cast.endswith('.nc'):
        try:
            cruise_data_nc = xa.load_dataset(sample_data_dir+cast)
            saldiff = cruise_data_nc.salinity_ch1 - cruise_data_nc.salinity_ch2
            print(f"{cast}: Profile Averaged Salinity Difference,STD (chan1-chan2){saldiff.mean().values},{saldiff.std().values}")        
            saldiff.plot.hist(bins=50,xlim=[-.005,.005])
        except:
            print(f"{cast}: less than two salinity channels")

In [15]:
# Following routines will eventually get ported to ecofocipy as subroutines to be called

import seawater as sw

def sigmat_update(salinity=None,temperature=None):
    '''
    Changes to T or S (commonly to despike values or apply a salinity offset) will need corresponding changes in sigmat
    '''
    # calculate sigmaT at 0db gauge pressure (s, t, p=0)
    sigt = (sw.eos80.dens0(s=salinity, t=temperature) - 1000)
    
    return sigt


## Generate Plots


### Make General Plots
- 1:1 plots for paired instruments for each cast (tells if a sensor failed)
- TS_Sigmat, Chlor/Par/Turb, Oxy,Temp
- T/S property property plot
- upcast/downcast plt

In [19]:
ctd_df

T_28   T2_35  rFv_971  \
time                depth lat       lon                                   
2010-08-28 07:29:00 0.0   59.891834 171.710831  8.5702  8.5747   0.0901   
                    1.0   59.891834 171.710831  8.5702  8.5747   0.0901   
                    2.0   59.891834 171.710831  8.5702  8.5747   0.0901   
                    3.0   59.891834 171.710831  8.5686  8.5735   0.0909   
                    4.0   59.891834 171.710831  8.5684  8.5724   0.0912   
...                                                ...     ...      ...   
                    63.0  59.891834 171.710831 -1.3218 -1.3173   0.0688   
                    64.0  59.891834 171.710831 -1.3219 -1.3163   0.0697   
                    65.0  59.891834 171.710831 -1.3205 -1.3152   0.0716   
                    66.0  59.891834 171.710831 -1.3203 -1.3150   0.0682   
                    67.0  59.891834 171.710831 -1.3174 -1.3137   0.0736   

                                                TRN_107  PAR_916  Chl_933  \
time                depth lat       lon                                     
2010-08-28 07:29:00 0.0   59.891834 171.710831   4.8285   1.1129   0.2168   
                    1.0   59.891834 171.710831   4.8285   1.1129   0.2168   
                    2.0   59.891834 171.710831   4.8285   1.1129   0.2168   
                    3.0   59.891834 171.710831   4.8285   0.9215   0.2264   
                    4.0   59.891834 171.710831   4.8265   0.7014   0.2299   
...                                                 ...      ...      ...   
                    63.0  59.891834 171.710831   4.1943   0.0000  -0.0380   
                    64.0  59.891834 171.710831   4.2049   0.0000  -0.0282   
                    65.0  59.891834 171.710831   4.1433   0.0000  -0.0053   
                    66.0  59.891834 171.710831   4.1230   0.0000  -0.0456   
                    67.0  59.891834 171.710831   4.1283   0.0000   0.0194   

                                                     PAR_905        O_65  \
time                depth lat       lon                                    
2010-08-28 07:29:00 0.0   59.891834 171.710831  6.735900e-01  278.578003   
                    1.0   59.891834 171.710831  6.735900e-01  278.578003   
                    2.0   59.891834 171.710831  6.735900e-01  278.578003   
                    3.0   59.891834 171.710831  4.476500e-01  278.921997   
                    4.0   59.891834 171.710831  2.578000e-01  278.667999   
...                                                      ...         ...   
                    63.0  59.891834 171.710831  1.000000e-12  277.494995   
                    64.0  59.891834 171.710831  1.000000e-12  276.790009   
                    65.0  59.891834 171.710831  1.000000e-12  277.260010   
                    66.0  59.891834 171.710831  1.000000e-12  277.102997   
                    67.0  59.891834 171.710831  1.000000e-12  276.588013   

                                                CTDOXY_4221     OST_62  \
time                depth lat       lon                                  
2010-08-28 07:29:00 0.0   59.891834 171.710831   289.009003  95.229759   
                    1.0   59.891834 171.710831   289.009003  95.229759   
                    2.0   59.891834 171.710831   289.009003  95.229759   
                    3.0   59.891834 171.710831   288.648010  95.348877   
                    4.0   59.891834 171.710831   288.592010  95.271339   
...                                                     ...        ...   
                    63.0  59.891834 171.710831   284.130005  74.936737   
                    64.0  59.891834 171.710831   283.690002  74.755074   
                    65.0  59.891834 171.710831   283.710999  74.881752   
                    66.0  59.891834 171.710831   284.053986  74.833961   
                    67.0  59.891834 171.710831   284.278015  74.699364   

                                                ATTN_55     Tr_904       S_41  \
time                depth lat       lon

In [23]:
calc_sigmat = True

for cast in sorted(os.listdir(sample_data_dir)):
    if cast.endswith('.nc'):
        cruise_data_nc = xa.load_dataset(sample_data_dir+cast)
        ctd_df = cruise_data_nc.to_dataframe()
        
        if calc_sigmat:
            #update sigmat (or calculate it I suppose)
            sigup = sigmat_update(salinity=ctd_df.S_41,
                                  temperature=ctd_df.T_28)
            ctd_df['sigma_t_ch1'] = sigup
            
            sigup2 = sigmat_update(salinity=ctd_df.S_42,
                                  temperature=ctd_df.T2_35)
            ctd_df['sigma_t_ch2'] = sigup2   
        sbe_p = sbe_ctd_plots.CTDProfilePlot(stylesheet='ggplot')
        plt,fig =sbe_p.plot3var(varname=['temperature_ch1','temperature_ch2','salinity_ch1','salinity_ch2','sigma_t_ch1','sigma_t_ch2'],
                          xdata=[ctd_df.T_28,ctd_df.T2_35,ctd_df.S_41,ctd_df.S_42,ctd_df.ST_70,ctd_df.ST_70],
                          ydata=ctd_df.index.get_level_values('depth'),
                          secondary=True,
                          xlabel=['Temperature','Salinity','Sigma-T'])

        DefaultSize = fig.get_size_inches()
        fig.set_size_inches( (DefaultSize[0], DefaultSize[1]*3) )
        plt.savefig(sample_data_dir+cast.replace('.nc','_TempSalSigmat.png'))
        plt.close(fig)

In [ ]:
for cast in sorted(os.listdir(sample_data_dir)):
    if cast.endswith('.nc'):
        cruise_data_nc = xa.load_dataset(sample_data_dir+cast)
        ctd_df = cruise_data_nc.to_dataframe()
        
        sbe_p = sbe_ctd_plots.CTDProfilePlot()
        plt,fig =sbe_p.plot2var(varname=['temperature_ch1','temperature_ch2','oxy_percentsat_ch1','oxy_percentsat_ch2'],
                          xdata=[ctd_df.temperature_ch1,ctd_df.temperature_ch2,ctd_df.oxy_percentsat_ch1,ctd_df.oxy_percentsat_ch2],
                          ydata=ctd_df.index.get_level_values('depth'),
                          secondary=True,
                          xlabel=['Temperature','Oxygen Saturation'])

        DefaultSize = fig.get_size_inches()
        fig.set_size_inches( (DefaultSize[0], DefaultSize[1]*3) )
        plt.savefig(sample_data_dir+cast.replace('.nc','_TempOxy.png'))
        plt.close(fig)

In [26]:
for cast in sorted(os.listdir(sample_data_dir)):
    if cast.endswith('.nc'):
        cruise_data_nc = xa.load_dataset(sample_data_dir+cast)
        ctd_df = cruise_data_nc.to_dataframe()
        
        sbe_p = sbe_ctd_plots.CTDProfilePlot(stylesheet='ggplot')
        plt,fig =sbe_p.plot2var(varname=['chlor_fluorescence','','par'],
                          xdata=[ctd_df.Chl_933,np.array([]),ctd_df.PAR_905,np.array([])],
                          ydata=ctd_df.index.get_level_values('depth'),
                          secondary=False,
                          xlabel=['Fluor','Par'])

        DefaultSize = fig.get_size_inches()
        fig.set_size_inches( (DefaultSize[0], DefaultSize[1]*3) )
        plt.savefig(sample_data_dir+cast.replace('.nc','_FluorPAR.png'))
        plt.close(fig)